In [22]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import os
os.chdir('/content/drive/My Drive/Colab Notebooks')

In [23]:
!ls

“cafai10-dnn.ipynb”的副本  ratings.csv		      Untitled0.ipynb
movielens_sample.txt	   suprise_recommder01.ipynb  使用surprise包.ipynb


In [4]:
!pip install surprise

     |████████████████████████████████| 6.5MB 4.4MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.0-cp36-cp36m-linux_x86_64.whl size=1678228 sha256=88467f03c34ba2d54cf0706462a3ad35c6360d2a724151619344a532fa33d70d
  Stored in directory: /root/.cache/pip/wheels/cc/fa/8c/16c93fccce688ae1bde7d979ff102f7bee980d9cfeb8641bcf
Successfully built scikit-surprise


使用surprise SVD系列算法推荐

In [0]:
from surprise import Dataset
from surprise import Reader
from surprise.prediction_algorithms import SVD ,SVDpp  
from surprise import accuracy
from surprise.model_selection import KFold
import pandas as pd
import  os

In [6]:
reader = Reader(line_format='user item rating', sep=',', skip_lines=1)
data = Dataset.load_from_file( "./ratings.csv" , reader = reader) 
from surprise.model_selection import train_test_split 

x_train , x_test = train_test_split( data ,test_size = 0.2 ,random_state = 10000 )
svd = SVD(biased= False) 
svd.fit(x_train)

prediction = svd.test(x_test)
accuracy.rmse( predictions=prediction )
# RMSE: 0.8548
# 0.8547798833361556

RMSE: 0.8543


0.8543026354117164

In [7]:
import pandas as pd 
import numpy as np
datas = pd.read_csv("ratings.csv" ,delimiter="," , skiprows=1  , names  =["user" ,"item" ,"rating"] ,usecols= [0,1,2] )
datas["user"] = datas["user"].astype(np.int32)
datas["item"] = datas["item"].astype(np.int32)
datas["rating"] = datas["rating"].astype(np.int32)
print(datas.dtypes )

user      int32
item      int32
rating    int32
dtype: object


In [8]:
reader = Reader(line_format='user item rating' )
data = Dataset.load_from_df( datas ,reader ) 
from surprise.model_selection import train_test_split 

x_train , x_test = train_test_split( data ,test_size = 0.2 ,random_state = 10000 )
svd = SVD(biased= False) 
svd.fit(x_train)

prediction = svd.test(x_test)
accuracy.rmse( predictions=prediction )

RMSE: 0.8883


0.888278927823158

In [9]:
from  collections import defaultdict
def get_top_n(predictions, n=10):
    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]
    return top_n

top_n = get_top_n(prediction, n=10)

# Print the recommended items for each user
for uid, user_ratings in top_n.items():
    print(uid, [iid for (iid, _) in user_ratings])


流式输出内容被截断，只能显示最后 5000 行内容。
2835 [1291, 1198, 110, 1250, 1193, 356, 3551, 4963, 1240, 2194]
5073 [590, 344, 4027, 592, 2671]
2239 [7153, 1136, 4246, 6333, 2671, 527, 2081, 1408, 4886, 5446]
2161 [104374, 96821, 3147, 5747, 1136, 91658, 1258, 6879, 3534, 4011]
124 [1136, 1617, 3793, 6874, 1265, 21, 1089, 8961, 4886, 4014]
5257 [1221, 608, 750, 1206, 1228, 1272, 318, 4848, 6711, 4262]
5672 [6, 349, 150, 592, 442, 434, 70, 19, 231, 413]
829 [77658, 44191, 48780, 1080, 6350, 60069, 55442, 41571, 87232, 1198]
534 [7153, 8132, 7502, 58559, 88125, 2571, 51662, 6539, 2028, 68954]
1682 [1147, 50, 1292, 1945, 1189, 3811, 2983, 150, 968, 1234]
3747 [2692, 1302, 296, 919, 3101, 4410, 5353]
1085 [1198, 2762, 34, 1270, 2804, 1304, 3114, 1396, 110, 1580]
5347 [3949, 27611, 27904, 4979, 47, 1884, 44191, 48774, 4011, 7099]
4930 [457, 480, 10, 329, 47, 165, 32, 34, 364, 367]
4705 [88163, 915, 1307, 4246, 2881, 5066, 7293, 56152, 4018, 32298]
5414 [1234, 908, 593, 1219, 916, 608, 527, 1035, 2081, 2908]
39

In [10]:
def precision_recall_at_k(predictions, k=10, threshold=3.5):
    '''Return precision and recall at k metrics for each user.'''

    # First map the predictions to each user.
    user_est_true = defaultdict(list)
    for uid, _, true_r, est, _ in predictions:
        user_est_true[uid].append((est, true_r))

    precisions = dict()
    recalls = dict()
    for uid, user_ratings in user_est_true.items():
        # Sort user ratings by estimated value
        user_ratings.sort(key=lambda x: x[0], reverse=True)
        # Number of relevant items
        n_rel = sum((true_r >= threshold) for (_, true_r) in user_ratings)
        # Number of recommended items in top k
        n_rec_k = sum((est >= threshold) for (est, _) in user_ratings[:k])
        # Number of relevant and recommended items in top k
        n_rel_and_rec_k = sum(((true_r >= threshold) and (est >= threshold))
                              for (est, true_r) in user_ratings[:k])
        # Precision@K: Proportion of recommended items that are relevant
        precisions[uid] = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 1
        # Recall@K: Proportion of relevant items that are recommended
        recalls[uid] = n_rel_and_rec_k / n_rel if n_rel != 0 else 1
    return precisions, recalls


precisions, recalls = precision_recall_at_k( prediction, k=5, threshold=4)

# Precision and recall can then be averaged over all users
print(sum(prec for prec in precisions.values()) / len(precisions))
print(sum(rec for rec in recalls.values()) / len(recalls))

0.9062555664932358
0.2292026617556219


In [0]:
# SVDpp 
svd = SVDpp() 
svd.fit(x_train)

prediction = svd.test(x_test)
accuracy.rmse( predictions=prediction )
top_n = get_top_n(prediction, n=10)

# Print the recommended items for each user
for uid, user_ratings in top_n.items():
  print(uid, [iid for (iid, _) in user_ratings])

In [0]:
# SVD
svd = SVD() 
svd.fit(x_train)

prediction = svd.test(x_test)
accuracy.rmse( predictions=prediction )
top_n = get_top_n(prediction, n=10)

# Print the recommended items for each user
for uid, user_ratings in top_n.items():
  print(uid, [iid for (iid, _) in user_ratings])

In [12]:
# 交叉验证
from surprise.model_selection.validation import cross_validate

algo =SVD(biased= False) 
res = cross_validate(algo, data, measures=['rmse', 'mae'], 
              cv= 3, return_train_measures=False ,
              verbose=False)
print(res )


{'test_rmse': array([0.90527692, 0.90347504, 0.90466677]), 'test_mae': array([0.69633793, 0.69525876, 0.69643319]), 'fit_time': (44.81118106842041, 45.26648163795471, 45.81706881523132), 'test_time': (4.129052400588989, 3.655106782913208, 3.666898488998413)}


In [0]:
# 网格搜索
# GrideSearchCV
from surprise.model_selection.search import GridSearchCV

param_grid = {'n_epochs': [5, 10], 'lr_all': [0.002, 0.005],
              'reg_all': [0.4, 0.6] , 'biased' :[False]          }


gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)
gs.fit(data)

# best RMSE score
print(gs.best_score['rmse'])
# combination of parameters that gave the best RMSE score
print(gs.best_params['rmse'])

# We can now use the algorithm that yields the best rmse:
model = gs.best_estimator['rmse']
model.fit(x_train)

predictions = model.predict(x_test)
top_n = get_top_n(predictions , n=10)

# Print the recommended items for each user
for uid, user_ratings in top_n.items():
  print(uid, [iid for (iid, _) in user_ratings])

使用xlearn

In [25]:
!pip install xlearn

     |████████████████████████████████| 4.9MB 4.7MB/s 





  Created wheel for xlearn: filename=xlearn-0.40a1-cp36-none-any.whl size=228713 sha256=09ab2c7c6057554b36606e7ad2770496fea0eec5a2c175e19421e88fb949448c
  Stored in directory: /root/.cache/pip/wheels/59/7d/71/699578f3cb69f16a2e5f648d978259dba959c92a5a6eca9451
Successfully built xlearn


In [0]:
# xlearn
import numpy as np
import xlearn as xl
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder 


In [0]:
import pandas as pd 
columns ="user_id,movie_id,rating,timestamp,title,genres,gender,age,occupation,zip".split(",")
df = pd.read_csv("movielens_sample.txt"  ,names = columns  ,skiprows=1)

In [0]:
df = df.drop( ["timestamp" ,"title" ],axis  =1 )
df  = df.genres.str.get_dummies(sep ="|").join(df).drop("genres" ,axis =1 )
df  = df.gender.str.get_dummies(sep ="|").join(df).drop("gender" ,axis =1 )

In [38]:
y  ,X = df.pop('rating') ,df
enc = LabelEncoder()
X['zip'] = enc.fit_transform(X['zip'])

X_train , X_test ,y_train ,y_test = train_test_split( X ,y , test_size =0.1)
# DMatrix transition
xdm_train = xl.DMatrix(X_train, y_train)
xdm_test = xl.DMatrix(X_test, y_test)



# Training task
fm_model = xl.create_fm()  # Use factorization machine
# we use the same API for train from file
# that is, you can also pass xl.DMatrix for this API now
fm_model.setTrain(xdm_train)    # Training data
fm_model.setValidate(xdm_test)  # Validation data

# 默认early_stoping ....  
param = {'task':'reg', 'lr':0.2, 
         'lambda':0.002, 
         'metric':'rmse' ,
         'epoch'  :10 , 
        'opt' :'sgd'}

fm_model.fit(param, './model_dm.out')

fm_model.setTest(xdm_test)  # Test data

# Start to predict
# The output result will be stored in output.txt
# if no result out path setted, we return res as numpy.ndarray

y_pred = fm_model.predict("./model_dm.out")
print(y_pred)



# error  module 'xlearn' has no attribute 'DMatrix' 
#  好像不可以 pip install 安装
# 源码安装成功运行


AttributeError: ignored